In [ ]:
%pip install -r requirements.txt

In [ ]:
# nohup python train_data_gen.py --audio_dir "/data/AAG/MTech_Project_Data/speech_data_filter"  --cnt 2500  --train_dir "train_data" --test_dir "test_data" --output_path "pkl_data"  --target_fs_values 16000 --clipping_thresholds 0.1 0.2 --time_clip 1 --win_len 500  --win_shift 125 --delta 300 --s_ratio 0.9 > data_gen_log.txt 2>&1 &

In [ ]:
# !python train_data_gen.py --audio_dir "/data/AAG/MTech_Project_Data/speech_data_filter" \
#     --cnt 10 --train_dir "train_data" --test_dir "test_data" --output_path "pkl_data" \
#     --target_fs_values 16000 --clipping_thresholds 0.1 0.2 --time_clip 1 --win_len 500 \
#     --win_shift 125 --delta 300 --s_ratio 0.9 

In [ ]:
# nohup python training.py --pkl_path pkl_data/training_data.pkl --epochs 500 --batch_size 512 --save_path saved_models --plot_path loss_plots --checkpoint_freq 50 --val_split 0.1 --resume --val > training_log.txt 2>&1 &

In [ ]:
# !python training.py --pkl_path pkl_data/training_data.pkl \
#     --epochs 4 \
#     --batch_size 256 \
#     --save_path saved_models \
#     --plot_path loss_plots \
#     --checkpoint_freq 2 \
#     --val_split 0.1 \
#     --resume 

In [ ]:
# !python evaluate.py --model_path "saved_models/final/complex_dft_unet_final.pth" \
#     --test_audio_dir "custom_sound" \
#     --output_dir "custom_output" \
#     --target_fs_values 16000 \
#     --clipping_thresholds 0.1 \
#     --time_clip 1 \
#     --factor 0.2 \
#     --eval_mode 1 \
#     --dynamic 0 \
#     --save 0

In [ ]:
# !python plot_loss_history.py --history_file saved_models/loss_history.json --output_path final_plots

In [3]:
import pickle

# Path to your .pkl file
file_path = "pkl_data/training_data.pkl"

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)